In [1]:
import os, sys, time
from time import sleep
from pathlib import Path
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import pyodbc
from openpyxl import Workbook, load_workbook
from openpyxl.drawing.image import Image
from openpyxl.utils.dataframe import dataframe_to_rows
import tempfile
from ttkthemes import ThemedStyle
import tkinter as tk
import tkinter.ttk as ttk
from tkinter import messagebox, filedialog, commondialog

In [2]:
class QC_Metrics_AvgTurnAround(): # {
    
    user_name = str(os.getlogin())
    outbound_dir = "C:/data/outbound/" + str(pd.Timestamp.now())[:10]
    desktop_dir = "OneDrive - Agilent Technologies/Desktop"
    
    def __init__(self, the_logger): # {
        self.the_logger = the_logger
        # Get/Set USERNAME & DESKTOP DIRECTORIES
        self.user_name_dir = os.path.join("C:/Users/", self.user_name)
        self.desktop_dir = os.path.join(self.user_name_dir, self.desktop_dir)
        print(self.user_name_dir)
        print(self.desktop_dir)
        self.run(day_range=30)
    # }
    
    def run(self, day_range): # {
        # TRY THE FOLLOWING
        try: # {
            # [2020-02-28]\\self.time_unit = time_unit
            # [2020-02-28]\\self.time_value = time_value
            self.day_range = day_range
            # get/set current date variable
            # [2020-03-06]\\the_date = pd.Timestamp.now()
            # [2020-03-11]\\the_date = pd.Timestamp(ts_input=str(self.end_date.get()))
            the_date = pd.Timestamp.now()
            # create variable for a month ago
            one_month_ago = the_date - timedelta(days = int(day_range))
            # [2020-02-28]\\
            ##one_month_ago = the_date - pd.Timedelta(unit=str(self.time_unit), value=str(self.time_value))
            print(type(one_month_ago))
            # CREATE METRICS TABLE FROM CLASS METHOD
            self.df_metrics_table = self.create_metrics_table()
            #############################
            # create .csv with no drops #
            #############################
            """
            self.df_metrics_table.to_csv(os.path.join(self.outbound_dir, "df_metrics_noDrop.csv"), 
                                         index=True)
            # DROP ROWS WITH PRODUCT LEVEL && QCDATE = NONE
            self.df_metrics_table.dropna(axis=0, subset=['QCDate', 'ProductLevel'], how='any', 
                                         inplace=True)
            # create .csv with DROPS
            self.df_metrics_table.to_csv(os.path.join(self.outbound_dir, "df_metrics_DROP.csv"), 
                                         index=True)
            """
            # Set index of METRICS
            # [2020-03-11]\\self.df_metrics_table.set_index(['QCDate', 'ProductLevel'], inplace=True)
            """
            self.df_metrics_table.index = self.df_metrics_table['QCDate']
            del self.df_metrics_table['QCDate']
            """
            """
            # [2020-03-11]\\del self.df_metrics_table['ProductLevel']
            print(self.df_metrics_table)
            # Display Index
            print(self.df_metrics_table.index)
            #############################
            # create .csv with UNSORTED #
            #############################
            self.df_metrics_table.to_csv(os.path.join(self.outbound_dir, "df_QC_metrics_UNSORTED.csv"), 
                                         index=True)
            # SORT INDEX
            self.df_metrics_table.sort_index(inplace=True)
            ###########################
            # create .csv with SORTED #
            ###########################
            self.df_metrics_table.to_csv(os.path.join(self.outbound_dir, "df_QC_metrics_SORTED.csv"), 
                                         index=True)
            print("SORTED:\n")
            print(self.df_metrics_table.index)
            """
            # SLICE & DICE THE DATAFRAME
            df_metrics_range = self.df_metrics_table[(self.df_metrics_table['QCDate'] > '2020-01-01') & (self.df_metrics_table['QCDate'] <= '2020-03-01')]
            # DETERMINE DAYS IN QC
            df_days_in_QC = pd.DataFrame(data=df_metrics_range['PfBatchID'])
            calculation = df_metrics_range['QCDate'] - df_metrics_range['AmpDate']
            df_days_in_QC['time-in-QC'] = calculation
            df_days_in_QC['ProductLevel'] = df_metrics_range['ProductLevel']
            df_days_in_QC.to_csv("C:/data/Outbound/2020-03-23/" 
                                 + str(pd.Timestamp.now())[:10] 
                                 + "-days-in-QC.csv")
            """
            { PERFORM GROUPBYS HERE }
            { PERFORM GROUPBY HERE }
            """
            # HOW MANY PfBatchID per ProductLevel, per QCDate?
            self.ProdsPerDay = df_metrics_range.groupby(['QCDate', 'ProductLevel'])[['PfBatchID']].count()
            df_prods_per_day = pd.DataFrame(data=self.ProdsPerDay)
            self.ProdsPerDay.to_csv("c:/data/Outbound/2020-03-23/"
                                    + str(pd.Timestamp.now())[:10]
                                    + "-ProdsPerDay.csv")
        # }
        except: # {
            errorMessage = str(sys.exc_info()[0]) + "\n"
            errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n\t\t"
            errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            typeE = str("TYPE : " + str(exc_type))
            fileE = str("FILE : " + str(fname))
            lineE = str("LINE : " + str(exc_tb.tb_lineno))
            messageE = str("MESG : " + "\n\n" + str(errorMessage) + "\n")
            print("\n" + typeE + 
                  "\n" + fileE + 
                  "\n" + lineE + 
                  "\n" + messageE)
        # }
        else: # {
            print("Operation Completed Successfully...")
        # }
    # }
    
    """
    Referred to as "ProdflowII" in SQL-Server
    """
    def pull_ProdflowII_table(self, table_name): # {
        # TRY THE FOLLOWING
        try: # {
            # CREATION CONNECTION STR
            conn_str = str(
                r'DRIVER={ODBC Driver 17 for SQL Server};'
                r'SERVER=wtkngappflow1.is.agilent.net;'
                r'DATABASE=ProdFlowII_Prod;'
                r'Trusted_Connection=yes;'
            )
            # CREATE PYODBC CONNECTION
            cnxn_ProdflowII = pyodbc.connect(conn_str)
            # [2020-02-028]\\crsr_ProdflowII = cnxn_ProdflowII.cursor()
            # PERFORM SQL QUERY AND SET AS DATAFRAME
            df_ProdflowII_table = pd.read_sql_query(sql='SELECT * FROM ' + str(table_name),
                                                    con=cnxn_ProdflowII
                                                    )
        # }
        except: # {
            errorMessage = str(sys.exc_info()[0]) + "\n"
            errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n\t\t"
            errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            typeE = str("TYPE : " + str(exc_type))
            fileE = str("FILE : " + str(fname))
            lineE = str("LINE : " + str(exc_tb.tb_lineno))
            messageE = str("MESG : " + "\n\n" + str(errorMessage) + "\n")
            print("\n" + typeE + 
                  "\n" + fileE + 
                  "\n" + lineE + 
                  "\n" + messageE)
        # }
        else: # {
            print("Operation Completed Successfully...")
            return df_ProdflowII_table
        # }
    # }
    
    """
    Referred to as "Prodflow" in SQL-Server
    """
    def pull_ProdflowIII_table(self, table_name): # {
        # TRY THE FOLLOWING
        try: # {
            # CREATE CONNECTION STRING
            conn_str = str(
                r'DRIVER={ODBC Driver 17 for SQL Server};'
                r'SERVER=wtkngappflow1.is.agilent.net;'
                r'DATABASE=ProdFlow;'
                r'Trusted_Connection=yes;'
            )
            # CREATE PYODBC CONNECTION
            cnxn_ProdflowIII = pyodbc.connect(conn_str)
            # [2020-02-28]\\crsr_ProdflowIII = cnxn_ProdflowIII,cursor()
            # PERFORM SQL QUERY AND SET AS DATAFRAME
            df_ProdflowIII_table = pd.read_sql_query(sql='SELECT * FROM ' + str(table_name),
                                                     con=cnxn_ProdflowIII,
                                                     parse_dates=['QCDate']
                                                     )
        # }
        except: # {
            errorMessage = str(sys.exc_info()[0]) + "\n"
            errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n\t\t"
            errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            typeE = str("TYPE : " + str(exc_type))
            fileE = str("FILE : " + str(fname))
            lineE = str("LINE : " + str(exc_tb.tb_lineno))
            messageE = str("MESG : " + "\n\n" + str(errorMessage) + "\n")
            print("\n" + typeE + 
                  "\n" + fileE + 
                  "\n" + lineE + 
                  "\n" + messageE)
        # }
        else: # {
            print("Operation Completed Successfully...")
            return df_ProdflowIII_table
        # }
    # }
    
    def create_metrics_table(self): # {
        # TRY THE FOLLOWING
        try: # {
            # pull PRODUCTS table
            self.df_products = self.pull_ProdflowII_table(table_name='Products')
            # RENAME ['Product#'] column to ["ProductNo"]
            self.df_products.rename(columns={'Product#': 'ProductNo'}, inplace=True)
            self.df_products.to_csv(os.path.join(self.outbound_dir, "df-PRODUCTS-table-" 
                                                + str(pd.Timestamp.now())[:10]
                                                + ".csv"), index=False)
            print("\tORDERS-TABLE:\n" + str(self.df_products.info()))
            # pull tblProdflow TABLE
            self.df_tblProdflow = self.pull_ProdflowIII_table(table_name='tblProdflow')
            self.df_tblProdflow.to_csv(os.path.join(self.outbound_dir, "df-tblProdflow-"
                                                    + str(pd.Timestamp.now())[:10] 
                                                    + ".csv"), index=False)
            print(self.df_tblProdflow.info())
            ########################
            # CREATE METRICS TABLE #
            ########################
            df_metrics_table = pd.merge(self.df_products, self.df_tblProdflow, on='ProductNo', how='right')
            ######################
            # CHANGE DATA TYPES: #
            ######################
            # CHANGE ['QCDate'] COLUMN TO DATETIME
            df_metrics_table['QCDate'] = pd.to_datetime(df_metrics_table['QCDate'])
            # SAME WITH OTHER COLUMNS
            df_metrics_table['AmpDate'] = pd.to_datetime(df_metrics_table['AmpDate'])
            df_metrics_table['OriginationDate'] = pd.to_datetime(df_metrics_table['OriginationDate'])
            df_metrics_table['EntryDate'] = pd.to_datetime(df_metrics_table['EntryDate'])
            # CHANGE ['ProductLevel'] COLUMN TO FLOAT
            # [2020-03-11]\\df_metrics_table['ProductLevel'] = df_metrics_table['ProductLevel'].values.astype(float)
            # [2020-03-11]\\df_metrics_table.ProductLevel = df_metrics_table.astype(float)
            # [2020-03-11]\\df_metrics_table['ProductLevel'] = df_metrics_table['ProductLevel'].values.astype(int)
            """
            # CHANGE 'ProductLevel' to CATEGORICAL
            df_metrics_table['ProductLevel'] = pd.Categorical(df_metrics_table['ProductLevel'],
                                                          categories=[1, 2, 3], ordered=False)
            """
            #####################################################
            # DROP ALL ROWS WITHOUT A 'QCDATE' & 'ProductLevel' #
            #####################################################
            df_metrics_table.dropna(axis=0, subset=['QCDate', 'ProductLevel'], 
                                    how='any', inplace=True)
        # }
        except: # {
            errorMessage = str(sys.exc_info()[0]) + "\n"
            errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n\t\t"
            errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            typeE = str("TYPE : " + str(exc_type))
            fileE = str("FILE : " + str(fname))
            lineE = str("LINE : " + str(exc_tb.tb_lineno))
            messageE = str("MESG : " + "\n\n" + str(errorMessage) + "\n")
            print("\n" + typeE + 
                  "\n" + fileE + 
                  "\n" + lineE + 
                  "\n" + messageE)
        # }
        else: # {
            print("Operation Completed Successfully...")
            return df_metrics_table
        # }
    # }
    
# }

In [3]:
if __name__ == "__main__": # {
    test_metrics = QC_Metrics_AvgTurnAround(the_logger=None)
# }

C:/Users/derbates
C:/Users/derbates\OneDrive - Agilent Technologies/Desktop
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
Operation Completed Successfully...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108544 entries, 0 to 108543
Data columns (total 42 columns):
ProductID          108544 non-null int64
ProductNo          108544 non-null object
ProductName        108516 non-null object
LineID             108544 non-null int64
Description        5976 non-null object
Specials           2443 non-null object
OriginationDate    108150 non-null datetime64[ns]
EntryDate          108542 non-null datetime64[ns]
Status             6036 non-null object
MatrixNumber       107659 non-null object
Components         108544 non-null int64
Availability       4843 non-null object
EquivalentSoln     103100 non-null object
EquivalentNeat     103258 non-null object
Packaging          107585 non-null object
Storage            100898 non-null object
ExpirationDate     108519 non-null float64
ShipTy